# Imports and Colab Mount

In [1]:
import datetime
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout, GRU, SimpleRNN, Input, LSTM, Dense, Bidirectional, Concatenate, Reshape, Lambda, Bidirectional
from keras.models import Model, Sequential
from keras import backend as K
from tensorflow.keras import layers
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import seaborn as sns

from numpy.random import seed
#from tensorflow import set_random_seed

%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
hourly = pd.read_excel("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/BASERECIFEhorario.xlsx", index_col=0, parse_dates=[["Data", "HORA (UTC)"]])
hourly = hourly[ ['Velocidade'] + [ col for col in hourly.columns if col != 'Velocidade' ] ]
quarto = int(hourly["Velocidade"].shape[0]/4)

resid_train = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_resid_train.csv").drop("Unnamed: 0", axis=1)
resid_test = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_resid_test2.csv").drop("Unnamed: 0", axis=1)

sarima_pred = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_prediction.csv").drop("Unnamed: 0", axis=1)
sarima_pred2 = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_prediction2.csv").drop("Unnamed: 0", axis=1)

#hourly["Velocidade"] = pd.concat([resid_train, resid_test]).reset_index()["0"].values
resid_series = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/recifehora_resid_series.csv").drop("Unnamed: 0", axis=1)

train_h = hourly.iloc[:quarto*2]
valid_h = hourly.iloc[quarto*2:quarto*3]
test_h = hourly.iloc[quarto*3:]

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [5]:
def make_data(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y

In [6]:
def make_data2(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

In [7]:
def make_data3(df, timestep, resid_check=False, resid=False, univariate=False):

  data = df.copy()

  if resid==True:
    data['Velocidade'] = resid_series.values

  if univariate==True:
    values = data["Velocidade"].values
    values = values.astype('float32')
    values = values.reshape(-1, 1)

    timestep = timestep
    n_features = 1
    n_obs = timestep * n_features
    reframed = series_to_supervised(values, timestep, 1)

  else:
    values = data.values
    values = values.astype('float32')
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #scaled = scaler.fit_transform(values)
    
    timestep = timestep
    n_features = data.shape[1]
    n_obs = timestep * n_features
    reframed = series_to_supervised(values, timestep, 1)
    reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_series.values[timestep:(quarto*2)+timestep]
    valid_y = resid_series.values[(quarto*2)+timestep:(quarto*3)+timestep]
    #test_y = resid_series.values[(quarto*3)+timestep:]
  if (resid==True):
    test_y = df['Velocidade'].values[(quarto*3)+timestep:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

# WanDB

In [8]:
!pip install wandb -qq
import wandb
from wandb.keras import WandbCallback
!wandb login

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 162 kB 53.4 MB/s 
     |████████████████████████████████| 181 kB 63.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 158 kB 73.6 MB/s 
     |████████████████████████████████| 157 kB 62.9 MB/s 
     |████████████████████████████████| 157 kB 58.0 MB/s 
     |████████████████████████████████| 157 kB 49.2 MB/s 
     |████████████████████████████████| 157 kB 53.3 MB/s 
     |████████████████████████████████| 157 kB 56.1 MB/s 
     |████████████████████████████████| 157 kB 46.8 MB/s 
     |████████████████████████████████| 157 kB 65.5 MB/s 
     |████████████████████████████████| 156 kB 58.8 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or pre

# Train

# RNN

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'RNN',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="recifehora_RESIDUAL_RNN")

Create sweep with ID: 48p0nggc
Sweep URL: https://wandb.ai/ancy/recifehora_RESIDUAL_RNN/sweeps/48p0nggc


In [ ]:
def create_rnn():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data3(hourly, config.timestep, resid_check=True, resid=True, univariate=True)
    model = Sequential()

    if layers > 1:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(SimpleRNN(units = num_units, dropout=dropout, return_sequences=True))

      model.add(SimpleRNN(units = num_units, dropout=dropout))

    else:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    resid_sum = (yhat_inv+sarima_pred.values[timestep:])

    print(sqrt(mean_squared_error(resid_sum, scaler_y.inverse_transform(test_y))))
    print(mean_absolute_error(resid_sum, scaler_y.inverse_transform(test_y)))
    print(mean_absolute_percentage_error(resid_sum, scaler_y.inverse_transform(test_y)))

    return model

In [ ]:
wandb.agent(sweep_id, create_rnn, count=50)

wandb: Agent Starting Run: u5lmwjjq with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 6
wandb: 	units: 64
wandb: Currently logged in as: ancy (use `wandb login --relogin` to force relogin)


0.7907551960942997
0.6411053073382728
0.22587488506550166



epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▂█▇▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,40
best_val_loss,0.00699
epoch,50
loss,0.0189
val_loss,0.007


wandb: Agent Starting Run: 7mwfqbsh with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.3
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 2
wandb: 	units: 64


0.7547355955545079
0.6172979137811836
0.2743235734992043



...

0.8057048936763646
0.654380943486631
0.22899857415438274



epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,▃█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,15
best_val_loss,0.00721
epoch,25
loss,0.0204
val_loss,0.00754


wandb: Agent Starting Run: mg6ffvgb with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 8
wandb: 	units: 256


0.814190906323112
0.6640435329600735
0.2318341991845708



epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,15
best_val_loss,0.00721
epoch,25
loss,0.0201
val_loss,0.00722


# LSTM

In [9]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'LSTM',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        'univariate': {
            'values': [False, True]
        },
        'resid': {
            'values': [False, True]
        }
        
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="recifehora_baseline")

Create sweep with ID: ycjej3xo
Sweep URL: https://wandb.ai/ancy/recifehora_baseline/sweeps/ycjej3xo


In [11]:
def create_lstm():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    'univariate':False,
    'resid': False,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data3(hourly, config.timestep, resid_check=True, resid=True, univariate=True)
    model = Sequential()

    if layers > 1:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(LSTM(units = num_units, dropout=dropout, return_sequences=True))

      model.add(LSTM(units = num_units, dropout=dropout))

    else:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_Loss 1": sqrt(mean_squared_error(resid_sum, scaler_y.inverse_transform(test_y)))})
    wandb.log({"Test_Loss 2": sqrt(mean_squared_error(resid_sum2, scaler_y.inverse_transform(test_y)))})

    return model

In [12]:
wandb.agent(sweep_id, create_lstm, count=10)

wandb: Agent Starting Run: mc2hop1n with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	layers: 8
wandb: 	lr: 0.1
wandb: 	resid: True
wandb: 	timestep: 12
wandb: 	units: 64
wandb: 	univariate: False
wandb: Currently logged in as: ancy. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
wandb: Adding directory to artifact (/content/wandb/run-20220927_050029-mc2hop1n/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050029-mc2hop1n/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050029-mc2hop1n/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050029-mc2hop1n/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050029-mc2hop1n/files/model-best)... Done. 0.1s
wandb: Adding 

Test_Loss 1,▁
Test_Loss 2,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.0
Test_Loss 1,1.04225
Test_Loss 2,0.66044
best_epoch,9
best_val_loss,0.02412
epoch,19


wandb: Agent Starting Run: chg5sndp with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0
wandb: 	layers: 1
wandb: 	lr: 0.01
wandb: 	resid: False
wandb: 	timestep: 3
wandb: 	units: 64
wandb: 	univariate: True


wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_050742-chg5sndp/files/model-bes

...

wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053424-fymnksu5/files/model-bes

Test_Loss 1,▁
Test_Loss 2,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.0
Test_Loss 1,1.07526
Test_Loss 2,0.65969
best_epoch,51
best_val_loss,0.02205
epoch,61


wandb: Agent Starting Run: 1lgj0xcl with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	layers: 1
wandb: 	lr: 0.01
wandb: 	resid: True
wandb: 	timestep: 1
wandb: 	units: 256
wandb: 	univariate: True


wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_053949-1lgj0xcl/files/model-bes

Test_Loss 1,▁
Test_Loss 2,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▂▄▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.0
Test_Loss 1,1.07837
Test_Loss 2,0.65785
best_epoch,21
best_val_loss,0.02231
epoch,31


# GRU

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'GRU',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="recifehora_RESIDUAL_GRU")

Create sweep with ID: ubqjlnjp
Sweep URL: https://wandb.ai/ancy/recifehora_RESIDUAL_GRU/sweeps/ubqjlnjp


In [ ]:
def create_gru():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, timestep, resid_check=True)
    model = Sequential()

    if layers > 1:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(GRU(units = num_units, dropout=dropout, return_sequences=True))

      model.add(GRU(units = num_units, dropout=dropout))

    else:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    lr = config.lr
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )
    
    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='val_loss',patience=5, min_lr=1e-6)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])

    print(sqrt(mean_squared_error(resid_sum, scaler_y.inverse_transform(test_y))))
    print(mean_absolute_error(resid_sum, scaler_y.inverse_transform(test_y)))
    print(mean_absolute_percentage_error(resid_sum, scaler_y.inverse_transform(test_y)))

    return model

In [ ]:
wandb.agent(sweep_id, create_gru, count=50)

wandb: Agent Starting Run: hlrdke10 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	layers: 1
wandb: 	lr: 0.1
wandb: 	timestep: 4
wandb: 	units: 64


0.8248587283554925
0.6673677413084929
0.27461373858811955



epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,25
best_val_loss,0.00989
epoch,35
loss,0.02548
val_loss,0.01007


wandb: Agent Starting Run: cwst2guv with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	layers: 4
wandb: 	lr: 0.1
wandb: 	timestep: 4
wandb: 	units: 64


0.816740719044784
0.6651484322819726
0.2290965691765442



...

0.8260633811062493
0.674785350737643
0.2342455448377584



epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,8
best_val_loss,0.00723
epoch,18
loss,0.01999
val_loss,0.0076


wandb: Agent Starting Run: 44k8w5v4 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.3
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 1
wandb: 	units: 128


0.7636298849622557
0.6184086549606715
0.22597705289370987



epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▅▃▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,17
best_val_loss,0.00645
epoch,27
loss,0.01839
val_loss,0.00686


# Transformer

In [13]:
sweep_config = {
    'name': 'New_Transformer',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'n_heads': {
            'values': [2, 4, 8]
        },
        'ff_dim': {
            'values': [128,256]
        },
        'n_blocks': {
            'values': [2, 4, 8]
        },
        'embed_dim': {
            'values': [32, 64, 128]
        },
        'time_2_vec': {
            'values': [2,3,4]
        }
    }
}

In [14]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="recifehora_baseline")

Create sweep with ID: xuucip8j
Sweep URL: https://wandb.ai/ancy/recifehora_baseline/sweeps/xuucip8j


In [15]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.ffn = keras.Sequential( [layers.Dense(ff_dim, activation = "gelu"), layers.Dense(feat_dim),] )
        self.layernorm1 = layers.BatchNormalization()
        self.layernorm2 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.embed_dim = embed_dim
        self.feat_dim = feat_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):

        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'feat_dim': self.feat_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate,
        })
        return config

In [16]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, kernel_size = 1):
        super(Time2Vec, self).__init__(trainable = True, name = 'Time2VecLayer')
        self.k = kernel_size

    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name = 'wb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        self.bb = self.add_weight(name = 'bb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        # periodic
        self.wa = self.add_weight(name = 'wa', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        self.ba = self.add_weight(name = 'ba', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)
        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1] * (self.k + 1)))
        return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * (self.k + 1))

    def get_config(self):

        config = super().get_config()
        config.update({
            'kernel_size': self.k,
        })
        return config

In [17]:
EPOCHS = 50
N_HEADS = 8
N_FOLDS = 10
FF_DIM = 256
N_BLOCKS = 6
EMBED_DIM = 64
BATCH_SIZE = 16
WINDOW_SIZE = 65
DROPUT_RATE = 0.0
TIME_2_VEC_DIM = 3
TRAIN_MODEL = True
SKIP_CONNECTION_STRENGTH = 0.9

In [18]:
def get_model(input_shape, time2vec_dim = 3):
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "selu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)
    return model

In [19]:
def get_model2():

    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'timestep': 3,
    'n_heads': 8,
    'ff_dim':256,
    'n_blocks':6,
    'embed_dim':64,
    'dropout':0,
    'time_2_vec':3,
    }

    wandb.init(config=config_defaults)
    config = wandb.config
    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data3(hourly, config.timestep, resid_check=True, resid=True, univariate=True)


    batch_size=config.batch_size
    lr=config.lr
    N_HEADS = config.n_heads
    FF_DIM = config.ff_dim
    N_BLOCKS = config.n_blocks
    EMBED_DIM = config.embed_dim
    DROPUT_RATE = config.dropout
    time2vec_dim = config.time_2_vec

    input_shape = train_X.shape[1:]
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "selu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
                  )

    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)]
              )
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_MSE": mean_squared_error(resid_sum2, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_RMSE": sqrt(mean_squared_error(resid_sum2, scaler_y.inverse_transform(test_y)))})

    wandb.log({"Test_MAE": mean_absolute_error(resid_sum2, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_MAPE": mean_absolute_percentage_error(resid_sum2, scaler_y.inverse_transform(test_y))})
    
    return model

In [20]:
wandb.agent(sweep_id, get_model2, count=10)

wandb: Agent Starting Run: 4ykkw4tj with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0
wandb: 	embed_dim: 64
wandb: 	ff_dim: 128
wandb: 	n_blocks: 4
wandb: 	n_heads: 2
wandb: 	time_2_vec: 3
wandb: 	timestep: 4


wandb: Adding directory to artifact (/content/wandb/run-20220927_054753-4ykkw4tj/files/model-best)... Done. 0.1s


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▂▂▃▄▅▅▆▇▇█
loss,█▄▂▂▂▁▁▁▁▁▁
val_loss,▁██▂▆▃▃▃▂▂▂
GFLOPs,2e-05
Test_MAE,0.56881
Test_MAPE,0.19445
Test_MSE,0.50822


wandb: Agent Starting Run: ufdrxgbv with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	embed_dim: 32
wandb: 	ff_dim: 128
wandb: 	n_blocks: 2
wandb: 	n_heads: 4
wandb: 	time_2_vec: 3
wandb: 	timestep: 12


wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_054843-ufdrxgbv/files/model-best)... Done. 0.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061141-265q9li9/files/model-best)... Done. 0.1s


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,█▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂█▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.0
Test_MAE,0.48627
Test_MAPE,0.19353
Test_MSE,0.43343


wandb: Agent Starting Run: msofluyi with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	embed_dim: 128
wandb: 	ff_dim: 256
wandb: 	n_blocks: 2
wandb: 	n_heads: 4
wandb: 	time_2_vec: 4
wandb: 	timestep: 6


wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s
wandb: Adding directory to artifact (/content/wandb/run-20220927_061257-msofluyi/files/model-best)... Done. 0.1s


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▂▁▁▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,5e-05
Test_MAE,0.48457
Test_MAPE,0.19413
Test_MSE,0.435


# Transformer 2

In [ ]:
from tensorflow.keras import layers

In [ ]:
sweep_config = {
    'name': 'Transformer',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0.1, 0.3, 0.5]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'head_size': {
            'values': [256, 128, 512]
        },
        'num_heads': {
            'values': [4, 8, 2]
        },
        'ff_dim': {
            'values': [4]
        },
        'num_blocks': {
            'values': [2, 4, 8]
        },
        'lr': {
            'values': [1e-1, 1e-2, 1e-3]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="Recifehora_oldtrans")

Create sweep with ID: muajowvf
Sweep URL: https://wandb.ai/ancy/Recifehora_oldtrans/sweeps/muajowvf


In [ ]:
def transformer_encoder(inputs, head_size=256, num_heads=4, ff_dim=4, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model():

    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'head_size': 256,
    'num_heads': 4,
    'ff_dim': 4,
    'num_blocks': 4
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, config.timestep, resid_check=True)


    num_transformer_blocks=config.num_blocks
    inputs = keras.Input(shape=train_X.shape[1:])
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size=config.head_size, num_heads=config.num_heads, ff_dim=config.ff_dim, dropout=config.dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    x = layers.Dense(config.units, activation="relu")(x)
    x = layers.Dropout(config.dropout)(x)
    outputs = layers.Dense(1)(x)

    lr = config.lr

    model = keras.Model(inputs, outputs)
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='val_loss',patience=5, min_lr=1e-6)])
    
      
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])

    print(sqrt(mean_squared_error(resid_sum, scaler_y.inverse_transform(test_y))))
    print(mean_absolute_error(resid_sum, scaler_y.inverse_transform(test_y)))
    print(mean_absolute_percentage_error(resid_sum, scaler_y.inverse_transform(test_y)))

    
    return keras.Model(inputs, outputs)

In [ ]:
wandb.agent(sweep_id, build_model, count=50)